In [25]:
import casadi as ca


# Define the problem variables
start_pos = 0
goal_pos = 10
num_timesteps = 20

# Create a CasADi optimization problem
opti = ca.Opti()

# Define the decision variables
pos = opti.variable(num_timesteps)
inputs = opti.variable(num_timesteps - 1)
# input_left = opti.variable()
# input_right = opti.variable()

# opti.subject_to(input_left == -1)
# opti.subject_to(input_right == 1)

# Define the constraints
opti.subject_to(pos[0] == start_pos)
for i in range(num_timesteps - 1):
    opti.subject_to(pos[i + 1] == pos[i] + (2 * inputs[i]) - 1)
    opti.subject_to(inputs[i] <= 2)
    # opti.subject_to(inputs[i] <= 1)
opti.subject_to(pos[-1] == goal_pos)

# Define the objective
opti.minimize(ca.sum1(inputs))

# Solve the problem
opti.solver("ipopt")
sol = opti.solve()

# Print the solution
if sol.stats()["success"]:
    print(f"Minimum time steps required: {sol.value(ca.sum1(inputs))}")
    print("Inputs per time step:")
    for i in range(num_timesteps-1):
        print(f"{int(sol.value(inputs[i]))}", end=", ")
    print(f"{int(sol.value(inputs[-1]))}")



This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       59
Number of nonzeros in inequality constraint Jacobian.:       19
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       39
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       21
Total number of inequality constraints...............:       19
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       19

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.00e+01 4.44e-16  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [1]:
from ortools.sat.python import cp_model

# Define the problem variables
start_pos = 0
goal_pos = 10
num_timesteps = 20
model = cp_model.CpModel()
positions = [model.NewIntVar(0, goal_pos, f"pos_{i}") for i in range(num_timesteps)]
inputs = [model.NewIntVar(0, 1, f"input_{i}") for i in range(num_timesteps - 1)]

# Define the constraints
model.Add(positions[0] == start_pos)
for i in range(num_timesteps - 1):
    model.Add(positions[i + 1] == positions[i] + 2 * inputs[i] - 1)
model.Add(positions[-1] == goal_pos)

# Define the objective
objective = model.NewIntVar(0, num_timesteps, "objective")
model.Add(objective == sum(inputs))
model.Minimize(objective)

# Solve the problem
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solution
if status == cp_model.OPTIMAL:
    print(f"Minimum time steps required: {solver.ObjectiveValue()}")
    print("Inputs per time step:")
    for i in range(num_timesteps - 1):
        print(f"{solver.Value(inputs[i])}", end=", ")
    print(f"{solver.Value(inputs[-1])}")